# Notebook pour réaliser l'extraction des posts et commentaires
* __Description__: Extraction des commentaires et des posts de Reddit pour leurs sauvegarde dans BigQuery
* __Source__: API de REDDIT
* __Output__: Table comment (BigQuery) et post
* __auteur__: 
* __date de création__: 14/09/2022
* __date de mise a jour__: 14/09/2022

### Import des outils

In [92]:
!mkdir /data
!gsutil rsync -r the-clean-project/notebooks/jupyter /data 

mkdir: cannot create directory ‘/data’: File exists
Building synchronization state...
Starting synchronization...
Copying gs://the-clean-project/.ipynb_checkpoints/Connect python-Copy1.ipynb/checkpoint...
Copying gs://the-clean-project/.ipynb_checkpoints/Connect python.ipynb/checkpoint...
Copying gs://the-clean-project/.ipynb_checkpoints/Extraction_reddit/Untitled.ipynb/checkpoint...
Copying gs://the-clean-project/.ipynb_checkpoints/Extraction_reddit/extraction_complete-Copy1.ipynb/checkpoint...
- [4 files][ 55.2 KiB/ 55.2 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m rsync ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://the-clean-project/.ipynb_checkpoints/Extraction_reddit/extraction_complete.ipynb/checkpoint...
Copying gs://the-clean-project/.ipynb_checkpoints/Extra

In [91]:
!ls /data

In [29]:
%run ../GCS/Extraction_reddit/libs/extraction_lib.ipynb

Exception: File `'../GCS/Extraction_reddit/libs/extraction_lib.ipynb.py'` not found.

## Extraction complète

Cette fonction permet de ...

### Extraction complète

In [ ]:
def extraction_complete(subreddit:str):
    """
        Fonction qui, quand on lui fournit le subreddit, va récupérer de façon automatique :
        - ses posts
        - les commentaires associés à chacun des posts
        Param:
            subreddit: str, nom du subreddit au format "nom_subreddit"
    """ 
    
    posts_extraction(subreddit)
    
    # Recup BigQuery
    date = date_extraction[:10]
    heure = date_extraction[11:]
    
    sql = f"SELECT id_post FROM `mimetic-coral-355913.dwh.post` WHERE extraction_utc = '{date}T{heure}'"
    
    # BigQuerry to Pandas DataFrame
    pandas_df = client.query(sql).to_dataframe()
    # Pandas DF to Spark DataFrame
    spark_df = spark.createDataFrame(pandas_df)
    # Spark DF to Spark RDD
    rdd = spark_df.rdd
    # flatmap pour faire qlqch
    rdd2 = rdd.map(lambda row : comments_extraction(f"r/{subreddit}", row["id_post"]))
    rdd3 = rdd2.reduce(lambda df1, df2: pd.concat([df1, df2]))
    
    insertionBigQuery('dwh', 'comment', rdd3)
    